In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import gpflow
from gpflow.kernels import Matern52 as mt52
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor

n_samples, n_iters = 100,100
n_data_points = 200

start = time.time()
X_train = np.random.rand(n_data_points,2)
y_train = np.random.rand(n_data_points,1)

def gpflow_sample(sample):
    variance = sample[:1][0]
    lengths = (sample[1:])
    kern = mt52(variance=variance, lengthscales=lengths)
    model = gpflow.models.GPR(
        (X_train, y_train),
        kernel=kern)

    mean, cov = model.predict_y(X_test, full_cov=False)
    mean = np.squeeze(mean.numpy())
    cov = np.squeeze(cov.numpy())

    return 0

start = time.time()

'''with ThreadPoolExecutor() as excecutor:
    for i in range(n_iters):
        X_test = np.random.rand(5,2)
        samples = np.random.rand(n_samples, 3)
        futures = [excecutor.submit(gpflow_sample, sample) for sample in samples]
        results = [future.result() for future in futures] # puts a wait condition on the threads finishing if you ask for the results here

        print(f'iter {i} done')'''

for i in range(n_iters):
        X_test = np.random.rand(5,2)
        samples = np.random.rand(n_samples, 3)
        futures = [gpflow_sample(sample) for sample in samples]

        print(f'iter {i} done')

end = time.time()

print(f'{n_samples* n_iters} trained equivalent to a {n_samples} sample, {n_iters} iteration smc sampler, this was done in {end-start:.4f} seconds')

iter 0 done


KeyboardInterrupt: 

it's definitely using gpu vram, but only 12% of the gpu might be able to multithread just point a join condition in each for loop, also doesn't seem to offloat the meme usage 17% at 200 data points instead of 20 - but basically the same speed for each iter, really just need to write a better proposal - also am curious to see the dif on normal windows vs - me trying to get it working on ubuntu with gpu usage - which i think is curr being used bc its quite hot (60C) idk tho


outputs:
* 10000 trained equivalent to a 100 sample, 100 iteration smc sampler, this was done in 323.0208 seconds - with GPU I think on 20 datapoints 5 test points
* 10000 trained equivalent to a 100 sample, 100 iteration smc sampler, this was done in 350 seconds - with GPU I think on 200 datapoints 5 test points

output on windows machine - same hardware as ubuntu just not with cuda working (~1% gpu usage):
* 10000 trained equivalent to a 100 sample, 100 iteration smc sampler, this was done in 421.3448 seconds - 200 datapoints 5 test points

so about 100 s quicker running on a gpu probably could be better if my gpu have more cuda cores - was actually for machine learning not for generating graphics, also multithreading doesn't speed this up on cpu or gpu - also crashed on gpu (maybe it ran out of vram memory or something although seemingly vram usage doesn't even change - i guess me converting everying to numpy to use is slowing down the code significantly as im moving stuff from ram to vram and back again all of the time)